## BBC - Project
### Data Exploration

In [1]:
DATA_PATH = './data/'

In [2]:
import numpy as np
from Bio import Geo

def load_geo(myfile):
    handle = open(myfile)
    records = Geo.parse(handle)
    return records
    
records = load_geo(DATA_PATH + 'GSE21510_family.soft')

The following cell should be executed multiple times to run through the data

In [3]:
nr = next(records)
print(nr)
print('\n---------------------- ENTITY_ATTRIBUTES:------------------------------\n')
print(nr.entity_attributes)

GEO Type: DATABASE
GEO Id: GeoMiame
Database_email: geo@ncbi.nlm.nih.gov

Database_institute: NCBI NLM NIH

Database_name: Gene Expression Omnibus (GEO)

Database_web_link: http://www.ncbi.nlm.nih.gov/geo

Column Header Definitions


---------------------- ENTITY_ATTRIBUTES:------------------------------

{'Database_name': 'Gene Expression Omnibus (GEO)', 'Database_institute': 'NCBI NLM NIH', 'Database_web_link': 'http://www.ncbi.nlm.nih.gov/geo', 'Database_email': 'geo@ncbi.nlm.nih.gov'}


In [11]:
# re-loading the data
records = load_geo(DATA_PATH + 'GSE21510_family.soft')
series_sample_id = []
sample_titles = []
genes = []
nb_cols = 0
nb_rows = 0
data = []
for r in records:
    rea = r.entity_attributes
    if 'Series_geo_accession' in rea:
        if rea['Series_geo_accession'] == 'GSE21510':
            series_sample_id = rea['Series_sample_id']
            nb_cols = len(series_sample_id)
    if 'Sample_title' in rea:
        sample_titles.append(rea['Sample_title'])
        if 'sample_table_begin' in rea:
            nb_rows = rea['Sample_data_row_count'] 
            data.append(r.table_rows)
                    

data will become a 2d numpy array of shape (54676, 149) == (NB_GENES+1, NB_SAMPLES+1).  
First row will contain the sample ids, first column will contain the ID_REFs of the genes.

In [12]:
all_data = np.ndarray((int(nb_rows)+1, int(nb_cols)+1), dtype=object)
# labels
all_data[0, 0] = 'ID_REF'
all_data[0, 1:] = np.array(series_sample_id)

for i, d in enumerate(data):
    values = np.array(d[1:])
    if (i == 0):
        all_data[1:, 0] = values[:, 0]
    all_data[1:, i+1] = values[:, 1]
data = np.array(all_data)

MemoryError: 

In [13]:
data.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
print(series_sample_id)
print('')
print(samples_geo_accessions)
print('')
print(sample_titles)
print('')
print(len(data), len(data[0]))
print(data[1][0])

In [ ]:
import re

samples = {}
patient_names = []
for i_t, title in enumerate(sample_titles):
    split = title.split(',')
    if(re.search('cancer', title)):
        samples[series_sample_id[i_t]] = 'cancer'
    else:
        samples[series_sample_id[i_t]] = 'normal'
        
    patient_names.append(split[0])
        
samples

In [10]:
print(sample_titles[104])

NameError: name 'sample_titles' is not defined

In [ ]:
# the keys of this dict are gene IDs, the values are GB_ACCs
genes_dict = {}
for i, g in enumerate(gene_ids[0]):
    if(i != 0): # do not inclide labels ('ID' & 'GB_ACC')
        genes_dict[g[0]] = g[1]

genes_dict

## Analyse lexicale

In [ ]:
!pip install nltk

In [4]:
import nltk
import os, codecs 
from nltk.text import TextCollection
from math import*


In [5]:
WORD_TO_SEARCH = ['cancer', 'colorectal', 'neoplasms', 'nonpolyposis', 'lynch' ]
query_vector = [1 for i in range(len(WORD_TO_SEARCH))]

LINE_SEPARATOR = "$$$"
CONETENT_SEPARATOR = ":::"
NEWLine = "\r\n"

In [6]:
data_path = "C:\\Users\\Jeremie Chatillon\\Documents\\HEIG\\BA-06\\BBC\\BBC-projet\\data\\"
data_file = "geneDefinition_dev.dat"
gen_id_file = "gen_id_dev.dat"
#data_file = "geneDefinition_prod.dat"

### Recherche des datas 

In [7]:
import pprint
import time
pp = pprint.PrettyPrinter(indent=2)
from Bio import Entrez
Entrez.API_KEY = 'b5ccd6f1153070941d8664573971486d8607'


#Entrez.email = "prenom.nom@heig-vd.ch" # une adresse email valide est nécessaire
Entrez.email = "jeremie.chatillon@heig-vd.ch" # une adresse email valide est nécessaire


from mpl_toolkits.basemap import Basemap # pour dessiner une carte du monde
import matplotlib.pyplot as plt # pour générer des graphiques
%matplotlib inline
#problème avec le module mpl_toolkits.basemap..
#esssai pour le récupérer
#j'ai mis a jour: 
#pip3.6 install --upgrade biopython --> pas fonctionné
#pip3 install --upgrade jupyter matplotlib numpy pandas scipy scikit-learn six Collecting jupyter
#pip3.6 install notebook --upgrade 
#conda install -c anaconda biopython


In [8]:
genes_dict = {'1007_s_at': 'U48705','1053_at': 'M87338'}

In [9]:
#gene_code = ['U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757','U48705', 'M87338', 'X51757']
#gene_code = ['U48705', 'M87338']

genes_dict = {'1007_s_at': 'U48705','1053_at': 'M87338'}
gene_code = []
gene_ids = []


for key in genes_dict:
    gene_code.append(genes_dict[key])
    gene_ids.append(key)
    

gen_ids = []

count = 0
while count < len(gene_code):
    try:
        gen_ids.append(Entrez.read(Entrez.esearch(db='gene', term=gene_code[count], retmode='xml'))['IdList'][0])
        count += 1
    except:
        time.sleep(3)

gen_ids

['780', '5982']

In [17]:
gen_ids
# Save gene id in file 
 
fd = codecs.open(data_path + gen_id_file, 'w', 'utf8')

str_ids = ""
for gen_id in gen_ids:
    str_ids += str(gen_id) + LINE_SEPARATOR
fd.write(str_ids)
fd.close()

NameError: name 'codecs' is not defined

In [18]:
# Read the file
fd = codecs.open(data_path + gen_id_file, 'r', 'utf8')
str_gen_ids = fd.read()
fd.close()

gen_ids = str_gen_ids.split(LINE_SEPARATOR)
gen_ids = gen_ids[:len(gen_ids)-1]
    
gen_ids

NameError: name 'codecs' is not defined

In [19]:
gen_datas_summary = []

count = 0
datas = [['ID', 'GB_ACC', 'text']]

while count < len(gen_ids):
    try:
        gb_gene = gen_ids[count]
        handle = Entrez.efetch(db="gene", id=gb_gene, retmode="xml")
        records = Entrez.read(handle)
        datas.append([gene_ids[count],gene_code[count],records[0]['Entrezgene_summary']])
        count += 1
    except:
        print("Sleep")
        time.sleep(3)



### Ecriture des sentences dans le fichier

In [20]:
fd = codecs.open(data_path + data_file, 'w', 'utf8')
for dat in datas:
    tmp = dat[0] + CONETENT_SEPARATOR + dat[1] + CONETENT_SEPARATOR + dat[2] + LINE_SEPARATOR
    fd.write(tmp)
fd.close()

NameError: name 'codecs' is not defined

### Lecture et analyse du fichier

In [22]:

fd = codecs.open(data_path + data_file, 'r', 'utf8')
war = fd.read()
fd.close()

NameError: name 'codecs' is not defined

In [ ]:

sents = war.split(LINE_SEPARATOR)
# Supression de la dernière ligne vide
sents = sents[:len(sents)-1]
id_gb_texts = []
for sent in sents:
    dat = sent.split(CONETENT_SEPARATOR)

    # Zone pour appliquer des fitres / lema / nomalisation
    dat[2] = dat[2].lower()
    id_gb_texts.append(dat)

len(id_gb_texts)

In [23]:
text_arr = []
for id_gb_text in id_gb_texts[1:]:
    text_arr.append(id_gb_text[2])
    
text_collection = TextCollection(text_arr)
len(text_arr)

NameError: name 'id_gb_texts' is not defined

In [24]:
tfidf_vector = [['id', 'tf-idf vector']]

for id_gb_text in id_gb_texts[1:]:
    vec = []
    for word in WORD_TO_SEARCH:
        tfidf = text_collection.tf_idf(word,id_gb_text[2] )
        vec.append(tfidf)
    tfidf_vector.append([id_gb_text[0], vec])
        
len(tfidf_vector)

NameError: name 'id_gb_texts' is not defined

In [25]:
#http://dataaspirant.com/2015/04/11/five-most-popular-similarity-measures-implementation-in-python/
def square_rooted(x):
    return round(sqrt(sum([a*a for a in x])),3)

def cosine_similarity(x,y):
    try:
        numerator = sum(a*b for a,b in zip(x,y))
        denominator = square_rooted(x)*square_rooted(y)
        return round(numerator/float(denominator),3)
    except:
        return 0.0
    

In [26]:
gene_similartites = [['id', 'cosSimilarity']]
for gene in tfidf_vector[1:]:
    dat = [gene[0], cosine_similarity(gene[1], query_vector)]
    gene_similartites.append(dat)
    
len(gene_similartites)
gene_similartites

[['id', 'cosSimilarity']]

In [27]:
gene_similartites_filtered1 = [['id', 'cosSimilarity']]
gene_similartites_filtered1.append(filter_zero(gene_similartites[1:]))
len(gene_similartites_filtered1[1])


NameError: name 'filter_zero' is not defined

In [ ]:
def take_n_max(arr):
    #todo
    return []